**Wikipedia spanish language model** 

In [ ]:
!git clone https://github.com/fastai/fastai2.git

In [ ]:
!pip install fastai2

In [ ]:
from google.colab import drive, files
import zipfile
import os
import re
from pathlib import Path
from fastai2.text.all import *
import dill as pickle

# Data preparation (Only run to create pickle file)

Connecting drive, train and test set organization, tokenization, numericalization and creation of LMDataLoader

In [ ]:
drive.mount('/content/drive/')

In [ ]:
#to not show cell output-- too long
%%capture
!unzip -q "/content/drive/My Drive/Colab Notebooks/TLAC/wiki_art.zip"

In [ ]:
def set_organization(dset):
  
  path="/content/wiki_art/{}".format(dset)

  for filename in os.listdir(path):
    os.rename(os.path.join(path, filename), os.path.join(path, re.sub('[^A-Za-z0-9]+(?![^.]*$)', '',filename)))

  n_files = len(list(os.listdir(path)))
  !ls $path
  print("elements_{}_set={} ".format(dset,n_files))


In [ ]:
%%capture
set_organization("train")

In [ ]:
%%capture
set_organization("test")

In [ ]:
path = Path('/content/wiki_art')
path.ls()

(#2) [Path('/content/wiki_art/test'),Path('/content/wiki_art/train')]

In [ ]:
texts = get_files(path, extensions=['.txt'], folders=['train', 'test'])

In [ ]:
def read_file(f): return L(f.read().split(' '))

In [ ]:
splits = RandomSplitter(valid_pct=0.1)(texts)
tfms = [Tokenizer.from_folder(path), Numericalize()]
dsets = Datasets(texts, [tfms], splits=splits, dl_type=LMDataLoader)

In [ ]:
pickle.dump(dsets, open(f'/content/dsets.pkl','wb'))

In [ ]:
!zip -r "/content/wiki_art_tok.zip" "/content/wiki_art_tok/"

In [ ]:
!mv "/content/dsets.pkl" "/content/drive/My Drive/Colab Notebooks/TLAC"

In [ ]:
!mv "/content/wiki_art_tok.zip" "/content/drive/My Drive/Colab Notebooks/TLAC"

# Language model



In [ ]:
drive.mount('/content/drive/')

In [ ]:
!unzip -q "/content/drive/My Drive/Colab Notebooks/TLAC/wiki_art_tok.zip"

In [ ]:
%%capture
!unzip /content/wiki_art_tok.zip

In [ ]:
!mv "/content/content/wiki_art_tok" "/content/"

In [ ]:
dsets = pickle.load(open(f'/content/drive/My Drive/Colab Notebooks/TLAC/dsets.pkl','rb'))

In [ ]:
dsets #wiki_art_tok

In [ ]:
bs,sl=64,80
dbunch_lm = dsets.dataloaders(bs=bs, seq_len=sl, val_bs=bs)

In [ ]:
dbunch_lm.show_batch()

,text,text_
0,"xxbos xxmaj xxunk xxmaj xxunk \n\n xxmaj abū ul - xxunk xxunk ibn xxmaj xxunk (; xxmaj granada , xxunk , octubre de 1485 ) fue el penúltimo sultán del xxmaj reino nazarí de xxmaj granada , llamado xxmaj xxunk xxmaj hasan o , por los cristianos , xxmaj xxunk xxmaj xxunk o xxmaj xxunk . \n\n xxmaj nacido en xxmaj granada en 1436 , creció bajo la protección de su padre , el sultán xxmaj abu xxmaj nasr xxmaj","xxmaj xxunk xxmaj xxunk \n\n xxmaj abū ul - xxunk xxunk ibn xxmaj xxunk (; xxmaj granada , xxunk , octubre de 1485 ) fue el penúltimo sultán del xxmaj reino nazarí de xxmaj granada , llamado xxmaj xxunk xxmaj hasan o , por los cristianos , xxmaj xxunk xxmaj xxunk o xxmaj xxunk . \n\n xxmaj nacido en xxmaj granada en 1436 , creció bajo la protección de su padre , el sultán xxmaj abu xxmaj nasr xxmaj xxunk"
1,"que intenta decir su primer nombre , siempre son interrumpidos por algún ruido . xxmaj el xxmaj sr . y la xxmaj sra . xxmaj turner creyeron que xxmaj xxunk sería niña y por ello , terminaron xxunk cosas de color rosado , incluyendo el gorro que haría característicos al personaje principal . \n\n xxmaj de acuerdo con un episodio ( "" una vida xxunk "" ) , si xxmaj xxunk no hubiera nacido , sus padres serían ultra millonarios","intenta decir su primer nombre , siempre son interrumpidos por algún ruido . xxmaj el xxmaj sr . y la xxmaj sra . xxmaj turner creyeron que xxmaj xxunk sería niña y por ello , terminaron xxunk cosas de color rosado , incluyendo el gorro que haría característicos al personaje principal . \n\n xxmaj de acuerdo con un episodio ( "" una vida xxunk "" ) , si xxmaj xxunk no hubiera nacido , sus padres serían ultra millonarios ,"
2,". \n\n xxmaj los conflictos con aspectos de naturaleza racista más importantes de la historia de xxmaj españa tienen una vinculación estrecha con las luchas religiosas de la xxmaj edad xxmaj media , aunque no se puede atribuir únicamente a la religión y tienen claras expresiones racistas . xxmaj para los reinos cristianos , son fundamentalmente la xxmaj reconquista y la xxmaj repoblación ( que puede entenderse como una limpieza étnica ) . xxmaj simultáneamente , en xxmaj al -","\n\n xxmaj los conflictos con aspectos de naturaleza racista más importantes de la historia de xxmaj españa tienen una vinculación estrecha con las luchas religiosas de la xxmaj edad xxmaj media , aunque no se puede atribuir únicamente a la religión y tienen claras expresiones racistas . xxmaj para los reinos cristianos , son fundamentalmente la xxmaj reconquista y la xxmaj repoblación ( que puede entenderse como una limpieza étnica ) . xxmaj simultáneamente , en xxmaj al - ándalus"
3,"diamante por la xxup riaa y vendió más de 30 millones de copias en todo el mundo , convirtiéndose en el álbum recopilatorio más vendido por un solista en la historia . « xxunk xxmaj my xxmaj love » alcanzó el número uno en xxmaj estados xxmaj unidos y los diez primeros lugares en otros países alrededor del mundo . xxmaj el video musical realizado para su promoción contaba con escenas de sadomasoquismo , "" bondage "" , besos entre","por la xxup riaa y vendió más de 30 millones de copias en todo el mundo , convirtiéndose en el álbum recopilatorio más vendido por un solista en la historia . « xxunk xxmaj my xxmaj love » alcanzó el número uno en xxmaj estados xxmaj unidos y los diez primeros lugares en otros países alrededor del mundo . xxmaj el video musical realizado para su promoción contaba con escenas de sadomasoquismo , "" bondage "" , besos entre personas"
4,"puente , después de atravesar 200 m de terreno abierto bajo fuego enemigo . xxmaj las fuerzas alemanas se retiraron de ambos lados del puente , el cual fue ocupado rápidamente por tanques irlandeses y el 2 . ° xxmaj batallón de la división . xxmaj el puente fue finalmente tomado por los xxmaj aliados después de cuatro días de combate . xxmaj la costosa operación fue llamada "" pequeña xxmaj omaha "" , en referencia a la playa",", después de atravesa

In [ ]:
learn = language_model_learner(dbunch_lm, AWD_LSTM, drop_mult=0.3)

In [ ]:
learn.lr_find()

In [ ]:
learn.fit_one_cycle(1, 1e-2)

In [ ]:
learn.save('wm1')